### This is part of the revision of the paper to look at the estimate genome size of certain Pst and Psh genomes


We will use jellyfish followed by GenomeScope for genome size estimates. We will use the following SRA samples correspoding to different publically available datasets. 


The corresponding short reads come from SRA as follows:
* SRR8208848 (Pst 93–210)
* SRR8208849 (Psh 93TX-2)
* SRX667385  (Pst 11/08) -- Warrior
* PRJEB15280 (Pst FW1) -- Kranich
* SRX7105378 (Pst DK0911)
* SRX3181917 (Pst 104E)
* SRR987409  (Pst 78)

Two additional isolates were provided directly from Dr. Diane Saunders.

* Pst-12/83
* Pst-12/86

These were downloaded with fastq-dumb 20200131 and 20200211

The following step need to be done:
* run jellyfish v2.2.6
* run genomescope 2 online http://qb.cshl.edu/genomescope/genomescope2.0/

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import os
import re
from Bio import SeqIO
from Bio import SeqUtils
import pysam
from Bio.SeqRecord import SeqRecord
from pybedtools import BedTool
import numpy as np
import pybedtools
import time
import matplotlib.pyplot as plt
import sys
import subprocess
import shutil
from Bio.Seq import Seq
import pysam
from Bio import SearchIO
import json
import glob
import scipy.stats as stats
import statsmodels as sms
import statsmodels.sandbox.stats.multicomp
import distance
import seaborn as sns
from pybedtools import BedTool
import matplotlib
from sklearn.externals.joblib import Parallel, delayed
import itertools as it
import tempfile
from scipy.signal import argrelextrema
import scipy
from het_cov import *
import importlib
import subprocess

/home/benjamin/anaconda3/lib/python3.5/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)


In [4]:
RAW_READ_DIR = '/home/benjamin/genome_assembly/Warrior/DK0911_v04/revisions_v01/raw_data'
JELLYFISH_DIR = '/home/benjamin/genome_assembly/Warrior/DK0911_v04/revisions_v01/genomesize/jellyfish'

In [3]:
!mkdir -p /home/benjamin/genome_assembly/Warrior/DK0911_v04/revisions_v01/genomesize/jellyfish

In [14]:
##get all the raw read data
fastqs = [x for x in os.listdir(RAW_READ_DIR) if x.endswith('.fastq') or x.endswith('.fq')]

In [15]:
samples = {x.split('_')[0] for x in fastqs} 
samples

{'ERR1607728',
 'S150',
 'SRR10406086',
 'SRR1533792',
 'SRR172670',
 'SRR6031720',
 'SRR8208848',
 'SRR8208849'}

In [18]:
jelly_k_cmd = 'jellyfish count -C -m 21 -s 1000000000 -t 20 %s_*.* -o %s.jf' 
jelly_h_cmd = 'jellyfish histo -t 20 %s.jf > %s.histo'
for sample in samples:
    cmd_1 = jelly_k_cmd % (os.path.join(RAW_READ_DIR, sample), os.path.join(JELLYFISH_DIR, sample))
    print(cmd_1)
    output = subprocess.run(cmd_1, shell=True)
    print(output.returncode)
    cmd_2 = jelly_h_cmd % (os.path.join(JELLYFISH_DIR, sample), os.path.join(JELLYFISH_DIR, sample))
    print(cmd_2)
    output = subprocess.run(cmd_2, shell=True)
    

jellyfish count -C -m 21 -s 1000000000 -t 20 /home/benjamin/genome_assembly/Warrior/DK0911_v04/revisions_v01/raw_data/SRR8208848_*.* -o /home/benjamin/genome_assembly/Warrior/DK0911_v04/revisions_v01/genomesize/jellyfish/SRR8208848.jf
0
jellyfish histo -t 20 /home/benjamin/genome_assembly/Warrior/DK0911_v04/revisions_v01/genomesize/jellyfish/SRR8208848.jf > /home/benjamin/genome_assembly/Warrior/DK0911_v04/revisions_v01/genomesize/jellyfish/SRR8208848.histo
jellyfish count -C -m 21 -s 1000000000 -t 20 /home/benjamin/genome_assembly/Warrior/DK0911_v04/revisions_v01/raw_data/SRR172670_*.* -o /home/benjamin/genome_assembly/Warrior/DK0911_v04/revisions_v01/genomesize/jellyfish/SRR172670.jf
0
jellyfish histo -t 20 /home/benjamin/genome_assembly/Warrior/DK0911_v04/revisions_v01/genomesize/jellyfish/SRR172670.jf > /home/benjamin/genome_assembly/Warrior/DK0911_v04/revisions_v01/genomesize/jellyfish/SRR172670.histo
jellyfish count -C -m 21 -s 1000000000 -t 20 /home/benjamin/genome_assembly/Warr

Files were uploaded to GenomeScope2 for analysis. This was last accessed online 20200212.